In [0]:
df = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load("/Volumes/pyspark_dbt/source/source_data/customers/customers.csv")


In [0]:
display(df)

In [0]:
schema_customers = df.schema
schema_customers

###  SPARK STREAMING 

In [0]:
df = spark.readStream.format("csv")\
    .option("header","true")\
    .schema(schema_customers)\
    .load("/Volumes/pyspark_dbt/source/source_data/customers/")


df.writeStream.format("delta")\
   .outputMode("append")\
   .option("checkpointLocation", "/Volumes/pyspark_dbt/bronze/checkpoint/customers")\
   .trigger(once = True)\
   .toTable("pyspark_dbt.bronze.customers")

### Loading and writing data to bronze table dynamically 

In [0]:
entities =["customers","drivers","locations","payments","trips","vehicles"]

In [0]:

for entity in entities:

  batch_read = spark.read.format("csv") \
    .option("header","true") \
    .option("inferSchema","true") \
    .load(f"/Volumes/pyspark_dbt/source/source_data/{entity}")

  schema_entity = batch_read.schema
  schema_entity

  df = spark.readStream.format("csv")\
  .option("header","true")\
  .schema(schema_entity)\
  .load(f"/Volumes/pyspark_dbt/source/source_data/{entity}/")

  df.writeStream.format("delta")\
  .outputMode("append")\
  .option("checkpointLocation", f"/Volumes/pyspark_dbt/bronze/checkpoint/{entity}")\
  .trigger(once = True)\
  .toTable(f"pyspark_dbt.bronze.{entity}")

### Displayed the data which is been stored in the broze environment as table

In [0]:
spark.table("pyspark_dbt.bronze.trips").show()